### IU Sketch Baseline 코드  
 - python모듈 설치 코드는 처음 한번 실행해주세요.     


In [ ]:
# !pip install imageio
# !pip install imageio --upgrade
# !pip install einops

In [ ]:
# 모듈 path 설정.
import os,sys
sys.path.insert(1, os.path.join(os.getcwd()  , '..'))

베이스 라인 코드.  

In [ ]:
import os, glob, random
import numpy as np
import matplotlib.pyplot as plt

import tensorflow.keras as keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model

import config_env as cfg

from classes.image_frame import ImgFrame
from models.dataset_generator import DataSetGenerator
from models.layer_conv import Conv2Plus1D, TConv2Plus1D
from models.layer_encoder import Encoder5D, Decoder5D
from models.layer_lstm import ConvLstmSeries

In [ ]:
# 필요한 디렉토리 없으면 생성.

# 학습용 raw_clip(gif) 파일 위치.
if not os.path.exists(cfg.RAW_CLIP_PATH):
    os.mkdir(cfg.RAW_CLIP_PATH)

# 모델 저장 위치
if not os.path.exists(cfg.MODEL_SAVE_PATH):
    os.mkdir(cfg.MODEL_SAVE_PATH)

# 임시 데이터 저장 위치
if not os.path.exists(cfg.TEMP_DATA_PATH):
    os.mkdir(cfg.TEMP_DATA_PATH)

In [ ]:
img_w, img_h = cfg.DATA_IMG_W, cfg.DATA_IMG_H
batch_size = cfg.DATA_BATCH_SIZE
time_steps = cfg.DATA_TIME_STEP

# encoder-decoder 모델 사용시
is_autoenc_model = True
train_val_ratio = 0.9

img_list = glob.glob(os.path.join(cfg.RAW_CLIP_PATH, "*.gif"))
train_img_cnt = int(len(img_list) * train_val_ratio)

random.shuffle(img_list)
train_img_list = img_list[:train_img_cnt]
val_img_list = img_list[train_img_cnt:]

tdgen = DataSetGenerator(imgs=train_img_list, batch_size=batch_size, time_step=time_steps, imgw=img_w, imgh=img_h, is_train=False, for_enc=is_autoenc_model)
vdgen = DataSetGenerator(imgs=val_img_list, batch_size=batch_size, time_step=time_steps, imgw=img_w, imgh=img_h, is_train=False, for_enc=is_autoenc_model)

enc_filter_count = 128
enc_conv_count = 3
lstm_out_filter_count = enc_filter_count*2**(enc_conv_count-1)
lstm_filter_count = 64

encoder = Encoder5D(enc_conv_count, enc_filter_count, (1, 3, 3), 2, "same")
decoder = Decoder5D(enc_conv_count, enc_filter_count, 1, (1, 3, 3), 2, "same")
lstms = ConvLstmSeries(lstm_filter_count, lstm_out_filter_count, [(3, 3), (3, 3), (3, 3)])

inputs = layers.Input(shape=(None, img_w, img_h, 1))
x = encoder(inputs)
if not is_autoenc_model:
    x = lstms(x)
x = decoder(x)
outputs = x

model = Model(inputs=inputs, outputs=outputs, name='sketcher')
model.compile(optimizer = keras.optimizers.Adam(1e-4), loss = 'binary_crossentropy')
model.summary()

plot_model(model, show_shapes=True, expand_nested=True, show_dtype=False)


In [ ]:
epoch_cnt = 100
steps_per_epoch = 50
val_steps = 2

history = model.fit(
            tdgen,
            validation_data = vdgen,
            steps_per_epoch = steps_per_epoch,
            validation_steps = val_steps,
            epochs = epoch_cnt,
            batch_size = batch_size,
            verbose = 1,
        )

In [ ]:
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='val')
plt.legend()
plt.show()

In [ ]:
encoder.seq.save(os.path.join(cfg.MODEL_SAVE_PATH, "enc_seq"))
decoder.seq.save(os.path.join(cfg.MODEL_SAVE_PATH, "dec_seq"))

if not is_autoenc_model:
    lstms.seq.save(os.path.join(cfg.MODEL_SAVE_PATH, "pred_seq"))

In [ ]:
# dataset중 하나만 예측에 입력
it = iter(tdgen)
x, y = next(it)
in_x = x[:1, :, :, :, :]
print(x.shape, y.shape, in_x.shape)

in_frm = ImgFrame(img=in_x[0][-1][:, :, :], do_norm=False)
in_img = in_frm.to_image()
plt.imshow(in_img, cmap='gray')

In [ ]:
pred = model.predict(in_x)
print(pred.shape)

frm = ImgFrame(img=pred[0][-1][:, :, :], do_norm=False)
img = frm.to_image()
# img.show()
plt.imshow(img, cmap='gray')